# Gradient Boosting Machine (GBM) Regressor - Dự đoán Tuổi thọ Trung bình

## Mục tiêu:
- Xây dựng mô hình Gradient Boosting Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập validation và test
- Lưu mô hình đã huấn luyện và trực quan hóa kết quả

## Giới thiệu
Gradient Boosting Machine (GBM) là một thuật toán ensemble learning mạnh mẽ, xây dựng mô hình theo cách tuần tự (sequential) thay vì song song như Random Forest. Mỗi cây mới được xây dựng để sửa chữa lỗi của các cây trước đó.

## Ưu điểm
- Hiệu suất dự đoán rất cao, thường vượt trội Random Forest.
- Xử lý tốt các mối quan hệ phi tuyến phức tạp.
- Tự động thực hiện chọn lọc đặc trưng.

## Nhược điểm
- Dễ bị overfitting.
- Thời gian huấn luyện lâu.
- Nhạy cảm với siêu tham số.
- Khó giải thích hơn các mô hình đơn giản.


## Bước 1 - Import các thư viện cần thiết

In [ ]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from datetime import datetime

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Thư viện để lưu mô hình
import joblib

# Cài đặt
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Đã import thành công tất cả thư viện!")

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [ ]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')
val_df = pd.read_csv('../data/processed/val.csv')
test_df = pd.read_csv('../data/processed/test.csv')

print("="*60)
print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập Train:      {train_df.shape}")
print(f"Kích thước tập Validation: {val_df.shape}")
print(f"Kích thước tập Test:       {test_df.shape}")
print("="*60)

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập Train:")
train_df.head()

## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`, và `year`.

In [ ]:
# Định nghĩa các cột đặc trưng (loại bỏ cột không cần thiết)
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code', 'year']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

X_val = val_df[feature_cols]
y_val = val_df['life_expectancy']

X_test = test_df[feature_cols]
y_test = test_df['life_expectancy']

print("="*60)
print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")
print(f"Kích thước X_val:   {X_val.shape}")
print(f"Kích thước y_val:   {y_val.shape}")
print(f"Kích thước X_test:  {X_test.shape}")
print(f"Kích thước y_test:  {y_test.shape}")
print("="*60)

## Bước 4 - Xây dựng mô hình GBM cơ bản

Trước tiên, xây dựng một mô hình Gradient Boosting cơ bản với tham số mặc định để làm baseline.

In [ ]:
# Tạo mô hình Gradient Boosting với tham số mặc định
gbm_baseline = GradientBoostingRegressor(random_state=42)

# Huấn luyện mô hình
print("Đang huấn luyện mô hình baseline...")
gbm_baseline.fit(X_train, y_train)
print("✓ Hoàn thành huấn luyện!")

# Dự đoán trên tập train và validation
y_train_pred_baseline = gbm_baseline.predict(X_train)
y_val_pred_baseline = gbm_baseline.predict(X_val)

# Tính toán các metrics
train_mae_baseline = mean_absolute_error(y_train, y_train_pred_baseline)
train_rmse_baseline = np.sqrt(mean_squared_error(y_train, y_train_pred_baseline))
train_r2_baseline = r2_score(y_train, y_train_pred_baseline)

val_mae_baseline = mean_absolute_error(y_val, y_val_pred_baseline)
val_rmse_baseline = np.sqrt(mean_squared_error(y_val, y_val_pred_baseline))
val_r2_baseline = r2_score(y_val, y_val_pred_baseline)

print("="*60)
print("KẾT QUẢ MÔ HÌNH BASELINE (tham số mặc định)")
print("="*60)
print(f"\nTập TRAIN:")
print(f"  MAE:  {train_mae_baseline:.4f} năm")
print(f"  RMSE: {train_rmse_baseline:.4f} năm")
print(f"  R²:   {train_r2_baseline:.4f}")

print(f"\nTập VALIDATION:")
print(f"  MAE:  {val_mae_baseline:.4f} năm")
print(f"  RMSE: {val_rmse_baseline:.4f} năm")
print(f"  R²:   {val_r2_baseline:.4f}")

print(f"\nSố lượng cây (estimators): {gbm_baseline.n_estimators}")
print(f"Learning rate: {gbm_baseline.learning_rate}")
print(f"Max depth: {gbm_baseline.max_depth}")
print("="*60)

# Nhận xét về overfitting
if train_r2_baseline - val_r2_baseline > 0.1:
    print("\n⚠️  Mô hình có dấu hiệu overfitting (R² train >> R² val)")
    print("→  Cần tối ưu hóa siêu tham số để giảm overfitting")
else:
    print("\n✓ Mô hình khá cân bằng, nhưng vẫn có thể cải thiện thêm")

## Bước 5 - Tối ưu hóa siêu tham số với GridSearchCV

Sử dụng GridSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình Gradient Boosting.

### Các siêu tham số cần tối ưu:
- **n_estimators:** Số lượng cây (boosting stages)
- **learning_rate:** Tốc độ học (shrinkage), kiểm soát đóng góp của mỗi cây
- **max_depth:** Độ sâu tối đa của mỗi cây
- **min_samples_split:** Số mẫu tối thiểu để chia một nút
- **min_samples_leaf:** Số mẫu tối thiểu ở một nút lá
- **subsample:** Tỷ lệ mẫu được sử dụng để huấn luyện mỗi cây (giúp giảm overfitting)

In [ ]:
# Định nghĩa không gian siêu tham số để tìm kiếm
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

print("="*60)
print("TÌM KIẾM SIÊU THAM SỐ TỐI ƯU")
print("="*60)
print(f"Số lượng tổ hợp tham số: {np.prod([len(v) for v in param_grid.values()])}")
print("Phương pháp: GridSearchCV với 5-Fold Cross-Validation")
print("Metric đánh giá: Negative RMSE (neg_root_mean_squared_error)")
print("\nĐang thực hiện tìm kiếm...")
print("⚠️  Lưu ý: Quá trình này có thể mất nhiều thời gian (15-45 phút)")
print("="*60)

# Tạo KFold với 5 folds
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Tạo GridSearchCV
grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=param_grid,
    cv=kfold,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

# Thực hiện tìm kiếm
grid_search.fit(X_train, y_train)

print("\n✓ Hoàn thành tìm kiếm!")

## Bước 6 - Phân tích kết quả GridSearchCV

Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [ ]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_gbm = grid_search.best_estimator_
best_params = grid_search.best_params_
best_cv_score = -grid_search.best_score_  # Chuyển về dương (RMSE)

print("="*60)
print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

print(f"\nRMSE Cross-Validation (5-fold): {best_cv_score:.4f} năm")
print("="*60)

# Thông tin về mô hình tối ưu
print(f"\nSố lượng cây (estimators): {best_gbm.n_estimators}")
print(f"Learning rate: {best_gbm.learning_rate}")
print(f"Max depth: {best_gbm.max_depth}")
print(f"Subsample: {best_gbm.subsample}")

## Bước 7 - Đánh giá mô hình tối ưu trên tập Validation

Sử dụng mô hình đã tối ưu để dự đoán trên tập validation và tính toán các metrics.

In [ ]:
# Dự đoán trên tập train và validation
print("Đang thực hiện dự đoán...")
y_train_pred = best_gbm.predict(X_train)
y_val_pred = best_gbm.predict(X_val)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# Tính toán các metrics cho tập validation
val_mae = mean_absolute_error(y_val, y_val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
val_r2 = r2_score(y_val, y_val_pred)

print("="*60)
print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print(f"\nTập TRAIN:")
print(f"  MAE:  {train_mae:.4f} năm")
print(f"  RMSE: {train_rmse:.4f} năm")
print(f"  R²:   {train_r2:.4f}")

print(f"\nTập VALIDATION:")
print(f"  MAE:  {val_mae:.4f} năm")
print(f"  RMSE: {val_rmse:.4f} năm")
print(f"  R²:   {val_r2:.4f}")

print("\n" + "="*60)
print("SO SÁNH VỚI BASELINE")
print("="*60)
print(f"Cải thiện RMSE trên validation: {val_rmse_baseline - val_rmse:.4f} năm")
print(f"Cải thiện R² trên validation:   {val_r2 - val_r2_baseline:.4f}")
print("="*60)

## Bước 8 - Phân tích Feature Importance

Phân tích mức độ quan trọng của từng đặc trưng trong việc dự đoán tuổi thọ trung bình.

In [ ]:
# Lấy feature importance
feature_importance = best_gbm.feature_importances_

# Tạo DataFrame để sắp xếp
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("="*60)
print("ĐỘ QUAN TRỌNG CỦA CÁC ĐẶC TRƯNG (Feature Importance)")
print("="*60)
for idx, row in importance_df.iterrows():
    print(f"{row['feature']:20s}: {row['importance']:.4f} {'█' * int(row['importance'] * 100)}")
print("="*60)

# Vẽ biểu đồ feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'], color='darkviolet', edgecolor='black')
plt.xlabel('Độ quan trọng', fontsize=12, fontweight='bold')
plt.ylabel('Đặc trưng', fontsize=12, fontweight='bold')
plt.title('Feature Importance - Gradient Boosting Regressor', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()

# Tạo thư mục nếu chưa có
os.makedirs('../visualization/gbm', exist_ok=True)
plt.savefig('../visualization/gbm/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ feature importance tại: ../visualization/gbm/feature_importance.png")

## Bước 9 - Trực quan hóa kết quả dự đoán

Vẽ biểu đồ so sánh giá trị thực tế và giá trị dự đoán trên tập validation.

In [ ]:
# Tạo figure với 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Subplot 1: Scatter plot - Predicted vs Actual (Validation)
axes[0].scatter(y_val, y_val_pred, alpha=0.5, color='darkviolet', edgecolor='black', linewidth=0.5)
axes[0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Dự đoán hoàn hảo')
axes[0].set_xlabel('Tuổi thọ thực tế (năm)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Tuổi thọ dự đoán (năm)', fontsize=12, fontweight='bold')
axes[0].set_title(f'Dự đoán vs Thực tế (Validation)\nR² = {val_r2:.4f}, RMSE = {val_rmse:.4f}', 
                  fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Subplot 2: Residual plot
residuals = y_val - y_val_pred
axes[1].scatter(y_val_pred, residuals, alpha=0.5, color='darkorange', edgecolor='black', linewidth=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Tuổi thọ dự đoán (năm)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals (Thực tế - Dự đoán)', fontsize=12, fontweight='bold')
axes[1].set_title('Biểu đồ Residuals (Validation)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../visualization/gbm/predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu biểu đồ dự đoán tại: ../visualization/gbm/predictions.png")

## Bước 10 - Phân tích Cross-Validation scores

Trực quan hóa phân bố điểm số RMSE từ 5-Fold Cross-Validation để đánh giá độ ổn định của mô hình.

In [ ]:
# Lấy kết quả cross-validation từ GridSearchCV
cv_results = grid_search.cv_results_
best_index = grid_search.best_index_

# Lấy scores của mô hình tốt nhất qua các folds
best_cv_scores = []
for i in range(5):  # 5 folds
    fold_score = -cv_results[f'split{i}_test_score'][best_index]  # Chuyển về RMSE dương
    best_cv_scores.append(fold_score)

print("="*60)
print("KẾT QUẢ 5-FOLD CROSS-VALIDATION")
print("="*60)
for i, score in enumerate(best_cv_scores, 1):
    print(f"Fold {i}: RMSE = {score:.4f} năm")

print(f"\nTrung bình: {np.mean(best_cv_scores):.4f} năm")
print(f"Độ lệch chuẩn: {np.std(best_cv_scores):.4f} năm")
print(f"Min: {np.min(best_cv_scores):.4f} năm")
print(f"Max: {np.max(best_cv_scores):.4f} năm")
print("="*60)

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
plt.plot(range(1, 6), best_cv_scores, marker='o', linewidth=2, markersize=10, color='darkviolet')
plt.axhline(y=np.mean(best_cv_scores), color='red', linestyle='--', linewidth=2, 
            label=f'Trung bình = {np.mean(best_cv_scores):.4f}')
plt.fill_between(range(1, 6), 
                 np.mean(best_cv_scores) - np.std(best_cv_scores),
                 np.mean(best_cv_scores) + np.std(best_cv_scores),
                 alpha=0.2, color='red', label=f'± 1 std = {np.std(best_cv_scores):.4f}')
plt.xlabel('Fold', fontsize=12, fontweight='bold')
plt.ylabel('RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('RMSE qua các Folds trong Cross-Validation', fontsize=14, fontweight='bold')
plt.xticks(range(1, 6))
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/gbm/cv_scores.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ CV scores tại: ../visualization/gbm/cv_scores.png")

# Đánh giá độ ổn định
if np.std(best_cv_scores) < 0.5:
    print("\n✓ Mô hình rất ổn định (độ lệch chuẩn < 0.5)")
elif np.std(best_cv_scores) < 1.0:
    print("\n✓ Mô hình khá ổn định (độ lệch chuẩn < 1.0)")
else:
    print("\n⚠️  Mô hình có độ biến động cao (độ lệch chuẩn ≥ 1.0)")

## Bước 11 - Phân tích Training Deviance (Loss)

Gradient Boosting cho phép theo dõi quá trình học qua từng iteration. Vẽ biểu đồ deviance (loss) để kiểm tra overfitting.

In [ ]:
# Lấy train score qua các iterations
train_score = best_gbm.train_score_

# Tính validation score qua các iterations
val_score = []
for i, pred in enumerate(best_gbm.staged_predict(X_val)):
    val_score.append(mean_squared_error(y_val, pred))

# Vẽ biểu đồ
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(train_score) + 1), train_score, label='Training Loss', linewidth=2, color='blue')
plt.plot(range(1, len(val_score) + 1), val_score, label='Validation Loss', linewidth=2, color='red')
plt.xlabel('Số lượng Boosting Iterations', fontsize=12, fontweight='bold')
plt.ylabel('Mean Squared Error (Loss)', fontsize=12, fontweight='bold')
plt.title('Training vs Validation Loss qua các Iterations', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/gbm/training_loss.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu biểu đồ training loss tại: ../visualization/gbm/training_loss.png")

# Phân tích
print("\n" + "="*60)
print("PHÂN TÍCH TRAINING LOSS")
print("="*60)
print(f"Training Loss (cuối): {train_score[-1]:.4f}")
print(f"Validation Loss (cuối): {val_score[-1]:.4f}")
print(f"Tổng số iterations: {len(train_score)}")

# Tìm iteration tốt nhất (validation loss thấp nhất)
best_iteration = np.argmin(val_score) + 1
print(f"\nIteration tốt nhất: {best_iteration}")
print(f"Validation Loss tại iteration tốt nhất: {val_score[best_iteration-1]:.4f}")

if best_iteration < len(train_score) * 0.8:
    print("\n⚠️  Validation loss đạt tối thiểu sớm → Có thể giảm n_estimators")
else:
    print("\n✓ Validation loss cải thiện đến cuối → Số iterations phù hợp")
print("="*60)

## Bước 12 - Đánh giá mô hình trên tập Test

Đánh giá hiệu suất cuối cùng của mô hình trên tập test (dữ liệu chưa từng thấy).

In [ ]:
# Dự đoán trên tập test
print("Đang dự đoán trên tập test...")
y_test_pred = best_gbm.predict(X_test)

# Tính toán các metrics cho tập test
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print("="*60)
print("KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST")
print("="*60)
print(f"\nMAE:  {test_mae:.4f} năm")
print(f"RMSE: {test_rmse:.4f} năm")
print(f"R²:   {test_r2:.4f}")
print("="*60)

# Tạo bảng so sánh
comparison_df = pd.DataFrame({
    'Tập dữ liệu': ['Train', 'Validation', 'Test'],
    'MAE': [train_mae, val_mae, test_mae],
    'RMSE': [train_rmse, val_rmse, test_rmse],
    'R²': [train_r2, val_r2, test_r2]
})

print("\nBẢNG SO SÁNH KẾT QUẢ:")
print(comparison_df.to_string(index=False))
print("="*60)

# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['MAE', 'RMSE', 'R²']
datasets = ['Train', 'Validation', 'Test']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, metric in enumerate(metrics):
    values = comparison_df[metric].values
    axes[idx].bar(datasets, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
    axes[idx].set_ylabel(metric, fontsize=12, fontweight='bold')
    axes[idx].set_title(f'{metric} trên các tập dữ liệu', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Thêm giá trị lên đầu cột
    for i, v in enumerate(values):
        axes[idx].text(i, v, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('../visualization/gbm/comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ so sánh tại: ../visualization/gbm/comparison.png")

## Bước 13 - Lưu mô hình đã huấn luyện

Lưu mô hình Gradient Boosting đã tối ưu vào file để sử dụng sau này.

In [ ]:
# Tạo thư mục saved_models/gbm nếu chưa có
os.makedirs('../saved_models/gbm', exist_ok=True)

# Lưu mô hình
model_path = '../saved_models/gbm/gradient_boosting.pkl'
joblib.dump(best_gbm, model_path)

print("="*60)
print("LƯU MÔ HÌNH")
print("="*60)
print(f"✓ Đã lưu mô hình tại: {model_path}")
print(f"Kích thước file: {os.path.getsize(model_path) / 1024:.2f} KB")
print("="*60)

# Kiểm tra tải lại mô hình
loaded_model = joblib.load(model_path)
y_test_pred_loaded = loaded_model.predict(X_test)
test_rmse_loaded = np.sqrt(mean_squared_error(y_test, y_test_pred_loaded))

print(f"\n✓ Kiểm tra tải lại mô hình:")
print(f"  RMSE trên test (mô hình gốc):      {test_rmse:.4f} năm")
print(f"  RMSE trên test (mô hình đã tải):   {test_rmse_loaded:.4f} năm")
print(f"  Chênh lệch: {abs(test_rmse - test_rmse_loaded):.10f} (rất nhỏ → OK)")

## Bước 14 - Phân tích sâu về Overfitting/Underfitting

Phân tích learning curve để hiểu rõ hơn về hành vi của mô hình.

In [ ]:
from sklearn.model_selection import learning_curve

# Tính toán learning curve
print("Đang tính toán learning curve (có thể mất vài phút)...")
train_sizes, train_scores, val_scores = learning_curve(
    best_gbm, X_train, y_train,
    cv=5,
    scoring='neg_root_mean_squared_error',
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1,
    random_state=42
)

# Chuyển sang RMSE dương và tính mean, std
train_scores_mean = -train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
val_scores_mean = -val_scores.mean(axis=1)
val_scores_std = val_scores.std(axis=1)

# Vẽ learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', linewidth=2, markersize=8, label='RMSE Train')
plt.plot(train_sizes, val_scores_mean, 'o-', color='red', linewidth=2, markersize=8, label='RMSE Validation')

plt.fill_between(train_sizes, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std,
                 alpha=0.2, color='blue')
plt.fill_between(train_sizes,
                 val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std,
                 alpha=0.2, color='red')

plt.xlabel('Số lượng mẫu huấn luyện', fontsize=12, fontweight='bold')
plt.ylabel('RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('Learning Curve - Gradient Boosting Regressor', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../visualization/gbm/learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Đã lưu learning curve tại: ../visualization/gbm/learning_curve.png")

# Phân tích
print("\n" + "="*60)
print("PHÂN TÍCH LEARNING CURVE")
print("="*60)
gap = train_scores_mean[-1] - val_scores_mean[-1]
if gap < 0.3:
    print("✓ Khoảng cách giữa train và validation RMSE rất nhỏ")
    print("  → Mô hình cân bằng rất tốt")
elif gap < 0.5:
    print("✓ Khoảng cách giữa train và validation RMSE nhỏ")
    print("  → Mô hình cân bằng tốt")
elif gap < 1.0:
    print("⚠ Khoảng cách giữa train và validation RMSE trung bình")
    print("  → Có dấu hiệu overfitting nhẹ")
else:
    print("✗ Khoảng cách giữa train và validation RMSE lớn")
    print("  → Mô hình bị overfitting, cần tăng regularization")
print("="*60)

## Bước 15 - Phân tích ảnh hưởng của Learning Rate

So sánh hiệu suất với các learning rate khác nhau để hiểu tác động của siêu tham số này.

In [ ]:
# Thử nghiệm với các learning rate khác nhau
learning_rates = [0.01, 0.05, 0.1, 0.2, 0.5]
lr_results = []

print("Đang thử nghiệm các learning rate khác nhau...")
for lr in learning_rates:
    gbm_lr = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=lr,
        max_depth=best_params['max_depth'],
        random_state=42
    )
    gbm_lr.fit(X_train, y_train)
    y_val_pred_lr = gbm_lr.predict(X_val)
    val_rmse_lr = np.sqrt(mean_squared_error(y_val, y_val_pred_lr))
    lr_results.append(val_rmse_lr)
    print(f"  Learning rate = {lr:.2f} → Validation RMSE = {val_rmse_lr:.4f}")

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
plt.plot(learning_rates, lr_results, marker='o', linewidth=2, markersize=10, color='darkviolet')
plt.axhline(y=val_rmse, color='red', linestyle='--', linewidth=2, 
            label=f'Best model RMSE = {val_rmse:.4f}')
plt.xlabel('Learning Rate', fontsize=12, fontweight='bold')
plt.ylabel('Validation RMSE (năm)', fontsize=12, fontweight='bold')
plt.title('Ảnh hưởng của Learning Rate đến Hiệu suất', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.tight_layout()
plt.savefig('../visualization/gbm/learning_rate_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Đã lưu biểu đồ learning rate analysis tại: ../visualization/gbm/learning_rate_analysis.png")

print("\n" + "="*60)
print("KẾT LUẬN VỀ LEARNING RATE")
print("="*60)
print("Learning rate nhỏ (0.01-0.05):")
print("  - Học chậm nhưng ổn định")
print("  - Cần nhiều iterations hơn")
print("  - Ít bị overfitting")
print("\nLearning rate trung bình (0.1-0.2):")
print("  - Cân bằng tốt giữa tốc độ và độ chính xác")
print("  - Thường cho kết quả tốt nhất")
print("\nLearning rate lớn (>0.5):")
print("  - Học nhanh nhưng không ổn định")
print("  - Dễ bỏ lỡ optimal solution")
print("  - Có thể bị overfitting")
print("="*60)

## Bước 16 - Tạo báo cáo tổng kết

Tổng hợp tất cả thông tin quan trọng về mô hình vào một báo cáo.

In [ ]:
# Tạo báo cáo
report = f"""
{'='*70}
           BÁO CÁO MÔ HÌNH GRADIENT BOOSTING REGRESSOR
{'='*70}

1. THÔNG TIN MÔ HÌNH:
   - Loại mô hình: Gradient Boosting Regressor
   - Thư viện: scikit-learn
   - Thời gian tạo: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

2. SIÊU THAM SỐ TỐI ƯU:
   - n_estimators: {best_params['n_estimators']}
   - learning_rate: {best_params['learning_rate']}
   - max_depth: {best_params['max_depth']}
   - min_samples_split: {best_params['min_samples_split']}
   - min_samples_leaf: {best_params['min_samples_leaf']}
   - subsample: {best_params['subsample']}

3. ĐẶC ĐIỂM MÔ HÌNH:
   - Số lượng boosting stages: {best_gbm.n_estimators}
   - Số đặc trưng: {len(feature_cols)}
   - Iteration tốt nhất: {best_iteration}

4. KẾT QUẢ ĐÁNH GIÁ:
   
   Tập TRAIN:
   - MAE:  {train_mae:.4f} năm
   - RMSE: {train_rmse:.4f} năm
   - R²:   {train_r2:.4f}
   
   Tập VALIDATION:
   - MAE:  {val_mae:.4f} năm
   - RMSE: {val_rmse:.4f} năm
   - R²:   {val_r2:.4f}
   
   Tập TEST (Final):
   - MAE:  {test_mae:.4f} năm
   - RMSE: {test_rmse:.4f} năm
   - R²:   {test_r2:.4f}

5. CROSS-VALIDATION (5-Fold):
   - RMSE trung bình: {np.mean(best_cv_scores):.4f} năm
   - Độ lệch chuẩn: {np.std(best_cv_scores):.4f} năm
   - RMSE tối thiểu: {np.min(best_cv_scores):.4f} năm
   - RMSE tối đa: {np.max(best_cv_scores):.4f} năm

6. SO SÁNH VỚI BASELINE:
   - Cải thiện RMSE validation: {val_rmse_baseline - val_rmse:.4f} năm
   - Cải thiện R² validation: {val_r2 - val_r2_baseline:.4f}

7. TOP 5 ĐẶC TRƯNG QUAN TRỌNG NHẤT:
"""

for i, (idx, row) in enumerate(importance_df.head(5).iterrows(), 1):
    report += f"   {i}. {row['feature']:20s}: {row['importance']:.4f}\n"

report += f"""
8. TRAINING LOSS ANALYSIS:
   - Training Loss (cuối): {train_score[-1]:.4f}
   - Validation Loss (cuối): {val_score[-1]:.4f}
   - Iteration tốt nhất: {best_iteration}/{len(train_score)}

9. KẾT LUẬN:
   - Mô hình Gradient Boosting đã được tối ưu hóa thành công
   - Hiệu suất trên tập test: RMSE = {test_rmse:.4f} năm, R² = {test_r2:.4f}
   - GBM thường cho hiệu suất cao hơn Random Forest nhờ học tuần tự
   - Mô hình ổn định với độ lệch chuẩn CV: {np.std(best_cv_scores):.4f}
   - Mô hình đã được lưu tại: {model_path}
   - Các biểu đồ đã được lưu tại: ../visualization/gbm/

10. ƯU ĐIỂM CỦA GRADIENT BOOSTING:
   - Hiệu suất dự đoán rất cao, thường tốt nhất trong các thuật toán truyền thống
   - Xử lý tốt mối quan hệ phi tuyến phức tạp
   - Tự động feature selection thông qua feature importance
   - Robust với outliers
   - Có thể theo dõi quá trình học qua training loss

11. NHƯỢC ĐIỂM VÀ LƯU Ý:
   - Thời gian huấn luyện lâu hơn Random Forest (sequential)
   - Dễ bị overfitting nếu không điều chỉnh cẩn thận
   - Nhạy cảm với siêu tham số (đặc biệt learning_rate và n_estimators)
   - Cần cân bằng giữa learning_rate và n_estimators

{'='*70}
"""

print(report)

# Lưu báo cáo vào file
report_path = '../saved_models/gbm/gradient_boosting_report.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"\n✓ Đã lưu báo cáo tại: {report_path}")

## Kết luận

### Tổng kết:
1. ✅ **Mô hình Gradient Boosting** đã được xây dựng và tối ưu hóa thành công
2. ✅ **GridSearchCV với 5-Fold CV** đã tìm được bộ siêu tham số tối ưu
3. ✅ **Feature Importance** đã xác định các yếu tố quan trọng nhất
4. ✅ **Training Loss Analysis** giúp hiểu quá trình học của mô hình
5. ✅ **Hiệu suất mô hình** đã được đánh giá toàn diện
6. ✅ **Mô hình đã được lưu** tại `saved_models/gbm/gradient_boosting.pkl`
7. ✅ **Trực quan hóa chi tiết** đã được tạo tại `visualization/gbm/`

### Điểm nổi bật của Gradient Boosting:
- **Sequential Learning:** Mỗi cây học từ lỗi của cây trước → Hiệu suất cao
- **Gradient Descent:** Tối ưu hóa loss function một cách có hệ thống
- **Learning Rate Control:** Kiểm soát tốc độ học giúp tránh overfitting
- **Stage-wise Training:** Có thể dừng sớm khi validation loss không cải thiện

### So sánh với các mô hình khác:
| Đặc điểm | Decision Tree | Random Forest | Gradient Boosting |
|----------|--------------|---------------|-------------------|
| Cách xây dựng | Một cây đơn | Nhiều cây song song | Nhiều cây tuần tự |
| Overfitting | Cao | Thấp | Trung bình |
| Hiệu suất | Thấp | Cao | Rất cao |
| Thời gian train | Nhanh | Trung bình | Chậm |
| Giải thích | Dễ | Khó | Khó |

### Khi nào nên dùng Gradient Boosting:
✓ Khi cần hiệu suất dự đoán cao nhất  
✓ Khi có đủ thời gian để tune hyperparameters  
✓ Khi dữ liệu có mối quan hệ phức tạp  
✓ Khi có kinh nghiệm xử lý overfitting  

### Khi nào KHÔNG nên dùng:
✗ Khi cần kết quả nhanh (thời gian train lâu)  
✗ Khi cần giải thích chi tiết (black box)  
✗ Khi dữ liệu quá nhỏ (dễ overfit)  
✗ Khi không có thời gian tune parameters  

### Bước tiếp theo:
- So sánh với XGBoost (cải tiến của GBM)
- So sánh với LightGBM và CatBoost
- So sánh với SVM Regression
- Xây dựng ensemble model kết hợp tất cả các thuật toán
- Phân tích chi tiết các trường hợp dự đoán sai